In [1]:
import gensim
import pandas as pd
import numpy as np
import sys
import os
import itertools
import time

import sklearn
from sklearn import cross_validation
from sklearn import svm
from sklearn import metrics

from scipy import stats
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans, MiniBatchKMeans

reload(sys)
sys.setdefaultencoding("UTF-8")

In [2]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [3]:
def clean_text(raw_text):
    review_text = BeautifulSoup(raw_text).get_text()
    words = review_text.lower().split()
    return(" ".join(words))

In [4]:
# Define a function to create bags of centroids
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [ ]:
# Getting a Glove Model
model_files = ["new.glove.twitter.27B.25d.txt", "new.glove.twitter.27B.50d.txt", 
               "new.glove.twitter.27B.100d.txt", "new.glove.twitter.27B.200d.txt"]
list_of_num_features = [25, 50, 100, 200]

languages = ["english"]
datafiles = ["summary-english-truth.txt"]
tasks = ["age", "gender"]
scoring_function = 'accuracy'
all_results = {}

In [ ]:
model_file = "new.glove.twitter.27B.25d.txt"
model = gensim.models.Word2Vec.load_word2vec_format(model_file,binary=False)

# ****** Run k-means on the word vectors and print a few clusters
#

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster

avg_word_per_cluster = 100
word_vectors = model.syn0
num_clusters = word_vectors.shape[0] / avg_word_per_cluster

# Initalize a k-means object and use it to extract centroids
print "Running K means"
kmeans_clustering = KMeans( n_clusters = num_clusters, n_jobs=cores )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."